In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from AnimalShelterModule import AnimalShelter



###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "scott"
as_test = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame(list(as_test.read({}, {"_id": 0})))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# image logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
# display image with link to www.snhu.edu
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                    style={'display':'block', 'margin-left':'auto', 'margin-right':'auto'})],
           href='https://www.snhu.edu'
          ),
# identifier
    html.Center(html.B(html.H1('Scott Smith SNHU CS-340 Dashboard'))),
    html.Hr(),
# radio buttons for filter options
    html.Div(className='row',
             style={'display': 'flex'},
                  children=[
                          dcc.RadioItems(
                                id='filter-type',
                                options=[
                                    {'label': 'Water Rescue', 'value': 'water'},
                                    {'label': 'Mountain Rescue', 'value': 'mountain'},
                                    {'label': 'Disaster Rescue', 'value': 'disaster'},
                                    {'label': 'Reset', 'value': 'reset'}
                                ],
                                labelStyle={'display': 'inline-block'}
                          )  
                  ]),
    html.Hr(),
# data table with sorting, filtering, paging, etc.
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# this callback filters the data as per customer requirements
@app.callback(
        [Output('datatable-id','data'),
         Output('datatable-id','columns'),
         Output('datatable-id', 'page_current')],
        [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        
        if filter_type == 'water':
               df = pd.DataFrame(list(as_test.read({"animal_type": "Dog", "breed": 
                    {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                    "sex_upon_outcome": "Intact Female", 
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}, {"_id": 0})))
        
        elif filter_type == 'mountain':
            df = pd.DataFrame(list(as_test.read({"animal_type": "Dog", "breed": {"$in": 
                    ["German Shepherd", "Alaskan Malamute", 
                     "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male", 
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}, {"_id": 0})))
        
        elif filter_type == 'disaster':
            df = pd.DataFrame(list(as_test.read({"animal_type": "Dog", "breed": {"$in": 
                    ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                     "Bloodhound", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male", 
                    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}, {"_id": 0})))
        
        elif filter_type == 'reset':
            df = pd.DataFrame(list(as_test.read({}, {"_id": 0})))
               
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        page_current=0      
        
        return (data,columns,page_current)

    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# draw pie graph showing ratio of breeds
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed')
    return [dcc.Graph(figure=fig)]

# draw geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Updated to show animal location
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app